In [1]:
# CELDA 0 ──────────────────────
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# CELDA 0.1 ───────── CONFIGURAR Y COMPROBAR GPU ──────────────────────────
import tensorflow as tf, subprocess, re, os

# 1. ¿Tenemos GPU?
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("⚠️  No hay GPU activa. Ve a 'Entorno de ejecución ▸ Cambiar tipo' y elige GPU.")
else:
    gpu = gpus[0]
    # 2. Información de la GPU (modelo y VRAM)
    details = tf.config.experimental.get_device_details(gpu)
    name = details.get('device_name', 'Desconocida')
    # Intenta sacar la memoria total con nvidia-smi
    mem_total = "?"
    try:
        smi = subprocess.check_output("nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits", shell=True)
        mem_total = smi.decode().strip().split('\n')[0] + " MiB"
    except Exception:
        pass
    print(f"✅  GPU detectada: {name}  –  Memoria total: {mem_total}")

    # 3. Memory-growth para no reservar toda la VRAM
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print("No se pudo establecer memory growth:", e)

    # 4. Activar mixed precision si es soportada
    from tensorflow.keras import mixed_precision
    if any(k in name for k in ("A100", "L4", "T4", "RTX", "V100", "P100")):
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print("🟢  Mixed precision FP16 activada (policy = mixed_float16)")
    else:
        print("ℹ️  Mixed precision no activada (GPU sin soporte FP16 o no detectada)")

# 5. Mostrar versión de TensorFlow y memoria que ve ahora
print("TensorFlow", tf.__version__)


✅  GPU detectada: Tesla T4  –  Memoria total: 15360 MiB
🟢  Mixed precision FP16 activada (policy = mixed_float16)
TensorFlow 2.18.0


In [3]:
# CELDA 1 ──────────────────────
import pandas as pd
from pathlib import Path

# Ruta base (AJÚSTALA)
BASE = Path("/content/drive/MyDrive/IABD/archive")
CSV  = BASE / "images_gender_final_v2.csv"

df = pd.read_csv(CSV)
print(f"Total de filas: {len(df)}")
display(df.head())


Total de filas: 5839


,image,sender_id,label,kids,gender
0,4285fab0-751a-4b74-8e9b-43af05deee22,124,Blouse,False,mujer
1,ea7b6656-3f84-4eb3-9099-23e623fc1018,148,T-Shirt,False,mujer
2,00627a3f-0477-401c-95eb-92642cbe078d,94,Blouse,False,mujer
3,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa,43,T-Shirt,False,hombre
4,3b86d877-2b9e-4c8b-a6a2-1d87513309d0,189,Shoes,False,mujer


Añadir columna filepath (para que Keras encuentre las imágenes)

In [4]:
# CELDA 2 ──────────────────────
from pathlib import Path

IMG_DIRS = [BASE/"images_original", BASE/"images_compressed", BASE/"images_new"]

def uuid_to_path(uuid):
    for d in IMG_DIRS:
        for ext in (".jpg", ".jpeg", ".png"):
            p = d / f"{uuid}{ext}"
            if p.exists():
                return str(p)
    return None

df["filepath"] = df["image"].apply(uuid_to_path)
df = df.dropna(subset=["filepath"])           # por si quedara alguna huérfana
print("Después de vincular rutas:", len(df))


Después de vincular rutas: 5839


Distribución de clases y asignación de grupos de data-augmentation

In [5]:
# CELDA 3 ───────── DISTRIBUCIÓN + GRUPOS + CLASES GLOBALES ────────────────
import numpy as np

vc = df["label"].value_counts().sort_values(ascending=False)
display(vc)

GRANDE, MEDIO, PEQUE = 400, 200, 100    # umbrales

def grupo(n):
    if n >= GRANDE: return "grande"
    if n >= MEDIO:  return "medio"
    if n >= PEQUE:  return "peque"
    return "minimo"

df["grupo"] = df["label"].map(lambda x: grupo(vc[x]))
display(df.groupby("grupo")["label"].count())

# 👉 CLASES GLOBALES (orden alfabético)
labels_global = sorted(df["label"].unique())
num_classes   = len(labels_global)
print("Número total de clases:", num_classes)


,count
label,
T-Shirt,1092
Longsleeve,710
Pants,655
Shoes,416
Shirt,405
Dress,351
Outwear,337
Shorts,318
Hat,246


,label
grupo,
grande,3278
medio,1670
minimo,219
peque,672


Número total de clases: 18


Generadores de Keras con data-augmentation por grupo

In [6]:
# CELDA 4 ───────── GENERADORES + CONCATSEQUENCE corregido ────────────────
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
import numpy as np

TARGET = (224, 224)
BATCH  = 32
SEED   = 42

def get_aug(grp):
    if grp == "grande":
        return ImageDataGenerator(rotation_range=10, horizontal_flip=True)
    if grp == "medio":
        return ImageDataGenerator(rotation_range=15,
                                  width_shift_range=0.1, height_shift_range=0.1,
                                  zoom_range=0.1, brightness_range=[0.8,1.2],
                                  horizontal_flip=True)
    if grp == "peque":
        return ImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.1, height_shift_range=0.1,
                                  zoom_range=0.15, shear_range=0.1,
                                  brightness_range=[0.7,1.3],
                                  horizontal_flip=True)
    return ImageDataGenerator(rotation_range=25,
                              width_shift_range=0.15, height_shift_range=0.15,
                              zoom_range=0.2, shear_range=0.15,
                              brightness_range=[0.6,1.4],
                              horizontal_flip=True)

flows = []
for grp in df["grupo"].unique():
    sub_df = df[df["grupo"] == grp]
    flow = get_aug(grp).flow_from_dataframe(
        sub_df,
        x_col="filepath",
        y_col="label",
        classes=labels_global,           # ← CLAVE: misma codificación
        class_mode="categorical",
        target_size=TARGET,
        batch_size=BATCH,
        shuffle=True,
        seed=SEED
    )
    flows.append(flow)

class ConcatSequence(Sequence):
    def __init__(self, sequences):
        self.seqs = sequences
        self.starts = np.cumsum([0] + [len(s) for s in sequences])

    def __len__(self):
        return self.starts[-1]

    def __getitem__(self, idx):
        seq_idx = np.searchsorted(self.starts, idx, side="right") - 1
        inner   = idx - self.starts[seq_idx]
        return self.seqs[seq_idx][inner]

train_seq       = ConcatSequence(flows)
steps_per_epoch = len(train_seq)
print("steps_per_epoch =", steps_per_epoch)


Found 1670 validated image filenames belonging to 18 classes.
Found 3278 validated image filenames belonging to 18 classes.
Found 219 validated image filenames belonging to 18 classes.
Found 672 validated image filenames belonging to 18 classes.
steps_per_epoch = 184


In [7]:
# CELDA 5 ───────── VALIDACIÓN estratificada + GENERADOR ──────────────────
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=42)

val_flow = ImageDataGenerator().flow_from_dataframe(
    val_df,
    x_col="filepath", y_col="label",
    classes=labels_global,            # ← usa la misma lista
    class_mode="categorical",
    target_size=TARGET,
    batch_size=BATCH,
    shuffle=False
)


Found 1168 validated image filenames belonging to 18 classes.


In [8]:
# CELDA 6 ─── CALCULAR class_weight ────────────────────────────────────────
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_int = df["label"].apply(lambda x: labels_global.index(x))
weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=y_int
)
class_weight = dict(enumerate(weights))
print("class_weight calculado para", num_classes, "clases")


class_weight calculado para 18 clases


In [9]:
# CELDA 7 ─── MODELO MobileNetV2 (fase 1 congelada) ───────────────────────
import tensorflow as tf
from tensorflow.keras import layers, models

base = tf.keras.applications.MobileNetV2(
    input_shape=TARGET + (3,),
    include_top=False,
    weights="imagenet")
base.trainable = False            # Fase 1: backbone congelado

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax',
                      dtype='float32')(x)     # fuerza FP32 si usas mixed precision

model = models.Model(base.input, output)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ cast[0][0]        │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis

 Total params: 2,923,090 (11.15 MB)

 Trainable params: 665,106 (2.54 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
# CELDA 8 ─── ENTRENAMIENTO FASE 1 (backbone congelado) ───────────────────
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=2),
    tf.keras.callbacks.ModelCheckpoint(
        BASE/"mobilenet_fase1.h5", save_best_only=True)
]

history1 = model.fit(
    train_seq,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=val_flow,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.1417 - loss: 3.2299 

184/184 ━━━━━━━━━━━━━━━━━━━━ 5184s 28s/step - accuracy: 0.1421 - loss: 3.2284 - val_accuracy: 0.1926 - val_loss: 2.5053 - learning_rate: 0.0010
Epoch 2/5
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3200 - loss: 2.1191

184/184 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.3199 - loss: 2.1201 - val_accuracy: 0.3784 - val_loss: 2.0647 - learning_rate: 0.0010
Epoch 3/5
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3742 - loss: 2.2427

184/184 ━━━━━━━━━━━━━━━━━━━━ 456s 2s/step - accuracy: 0.3742 - loss: 2.2421 - val_accuracy: 0.3818 - val_loss: 1.9686 - learning_rate: 0.0010
Epoch 4/5
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3913 - loss: 2.1169

184/184 ━━━━━━━━━━━━━━━━━━━━ 461s 3s/step - accuracy: 0.3915 - loss: 2.1165 - val_accuracy: 0.4332 - val_loss: 1.8741 - learning_rate: 0.0010
Epoch 5/5
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4343 - loss: 2.1951

184/184 ━━━━━━━━━━━━━━━━━━━━ 512s 3s/step - accuracy: 0.4343 - loss: 2.1932 - val_accuracy: 0.4443 - val_loss: 1.7990 - learning_rate: 0.0010


In [12]:
# CELDA 9 ─── ENTRENAMIENTO FASE 2 (fine-tune) ────────────────────────────
for layer in base.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_f2 = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=2),
    tf.keras.callbacks.ModelCheckpoint(
        BASE/"mobilenet_fase2.h5", save_best_only=True)
]

history2 = model.fit(
    train_seq,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=val_flow,
    class_weight=class_weight,
    callbacks=callbacks_f2,
    verbose=1
)


Epoch 1/10
151/184 ━━━━━━━━━━━━━━━━━━━━ 1:16 2s/step - accuracy: 0.8791 - loss: 0.4068

KeyboardInterrupt: 

In [13]:
# CELDA 10 ─── REPORT DE CLASIFICACIÓN ────────────────────────────────────
from sklearn.metrics import classification_report
import numpy as np

pred_prob = model.predict(val_flow, verbose=0)
pred_int  = pred_prob.argmax(axis=1)
true_int  = val_flow.classes

print(classification_report(true_int, pred_int, target_names=labels_global))


              precision    recall  f1-score   support

      Blazer       0.78      0.29      0.42        24
      Blouse       0.83      0.46      0.59        41
        Body       1.00      0.13      0.24        15
       Dress       0.42      0.47      0.45        70
         Hat       0.71      0.71      0.71        49
      Hoodie       0.62      0.41      0.49        39
  Longsleeve       0.49      0.62      0.54       142
       Other       0.00      0.00      0.00        10
     Outwear       0.28      0.72      0.40        68
       Pants       0.70      0.85      0.77       131
        Polo       0.75      0.38      0.50        40
       Shirt       0.86      0.37      0.52        81
       Shoes       0.92      0.67      0.78        83
      Shorts       0.44      0.75      0.56        64
       Skirt       0.45      0.57      0.51        42
     T-Shirt       0.76      0.50      0.60       219
         Top       0.40      0.21      0.28        19
  Undershirt       0.34    

In [14]:
# CELDA 11 ─── GUARDAR MODELO Y LABEL MAP ─────────────────────────────────
import json
model.save(BASE/"mobilenet_final.h5")

with open(BASE/"label_map.json", "w") as fp:
    json.dump(labels_global, fp)

print("Modelo y label_map guardados en:", BASE)


Modelo y label_map guardados en: /content/drive/MyDrive/IABD/archive
